In [4]:
!pip install krippendorff
!pip install openpyxl



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\lcopo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
     ---- -------------------------------- 30.7/250.0 kB 262.6 kB/s eta 0:00:01
     ----------------- ------------------ 122.9/250.0 kB 804.6 kB/s eta 0:00:01
     -------------------------------------  245.8/250.0 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\lcopo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## A: 2 Annotators, No missing data

In [6]:
import pandas as pd
from krippendorff import alpha

data = pd.read_excel('table1.xlsx')

# drop the first column > timestamps (not used)
data_cleaned = data.drop(data.columns[0], axis=1)
data_transposed = data_cleaned.transpose()

data_matrix = [data_transposed[0].tolist(), data_transposed[1].tolist()]

# metric application
alpha_scalar = alpha(reliability_data=data_matrix, level_of_measurement='interval')
print("Agreement:", alpha_scalar)


Agreement: 0.5757753071971914


## B: 3 Annotators, No missing data

In [7]:
data = pd.read_excel('table2.xlsx')

# drop the first column > timestamps (not used)
data_cleaned = data.drop(data.columns[0], axis=1)
data_transposed = data_cleaned.transpose()

data_matrix = [data_transposed[0].tolist(), data_transposed[1].tolist(), data_transposed[2].tolist()]

# metric application
alpha_scalar = alpha(reliability_data=data_matrix, level_of_measurement='interval')
print("Agreement:", alpha_scalar)


Agreement: 0.6373626373626373
